<a href="https://colab.research.google.com/github/sajal1302/Study-Material/blob/master/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

     |████████████████████████████████| 212.3MB 72kB/s 
     |████████████████████████████████| 204kB 41.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=d65341cf9cfb30967b0a8b7ffda3053501f8c5484ab9eabf589d5f994a593d74
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import logging
import json
logger = logging.getLogger('BCG-Use-Case')
logger.setLevel(logging.INFO)

In [3]:
def get_spark_session():
    spark = SparkSession.builder.appName("BCG-Project").enableHiveSupport().getOrCreate()
    return spark

In [ ]:
config = { 'files' : ['Charges_use.csv' ,
                      'Damages_use.csv' ,
                      'Endorse_use.csv',
                      'Primary_Person_use.csv' ,
                      'Restrict_use.csv' ,
                      'Units_use.csv'] ,
          'files_loc' : '',
          
          'feeds':{
              'Charges_use': {
                  'feed_name' : 'Charges_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Damages_use': {
                  'feed_name' : 'Damages_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Endorse_use': {
                  'feed_name' : 'Damages_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Primary_Person_use': {
                  'feed_name' : 'Primary_Person_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Restrict_use': {
                  'feed_name' : 'Restrict_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              },
              'Units_use': {
                  'feed_name' : 'Units_use.csv',
                  'feed_location' : '',
                  'delimeter' : '',
                  'compression' : '',
                  
              }
          }  
    
}

In [4]:
spark = get_spark_session()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
spark

In [8]:
! ls drive/MyDrive/bcg/staging_files/

Charges_use.csv  Endorse_use.csv	 Restrict_use.csv
Damages_use.csv  Primary_Person_use.csv  Units_use.csv


In [ ]:
! ls bcg/config

config.json


In [9]:
def load_json(file_path):
  with open(file_path) as f:
    config = json.load(f)
    return config

In [12]:
bcg_conf = load_json('/content/drive/MyDrive/bcg/config/config.json')

In [13]:
def load_file(bcg_conf , entity_name):
  try :
    feed_name = bcg_conf['feeds'][entity_name]['feed_name']
    file_loc = bcg_conf['feeds'][entity_name]['feed_location']
    file_format = bcg_conf['feeds'][entity_name]['file_format']
    delimeter = bcg_conf['feeds'][entity_name]['delimeter']
    header = bcg_conf['feeds'][entity_name]['header']
    full_path = file_loc + feed_name

    print(feed_name , file_loc , file_format, full_path )
    df_ = spark.read.load(full_path, 
                    format= file_format, sep= delimeter, inferSchema="true",
                    header=header)
    return df_

  except:
    logger.error("Check the Config for the Entity : {} ".format(entity_name))

In [14]:
def trim_columns(df):
  df_ = df.select([ trim(col(colm)).alias(colm.strip()) if dtype =='string' else col(colm).alias(colm.strip()) for colm,dtype in df.dtypes ])
  return df_

In [18]:
charges_use_df = trim_columns(load_file(bcg_conf , 'Charges_use'))
damages_use_df = trim_columns(load_file(bcg_conf , 'Damages_use'))
endorse_use_df = trim_columns(load_file(bcg_conf , 'Endorse_use'))
primary_person_use_df = trim_columns(load_file(bcg_conf , 'Primary_Person_use'))
restrict_use_df = trim_columns(load_file(bcg_conf , 'Restrict_use'))
units_use_df = trim_columns(load_file(bcg_conf , 'Units_use'))

Charges_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Charges_use.csv
Damages_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Damages_use.csv
Damages_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Damages_use.csv
Primary_Person_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Primary_Person_use.csv
Restrict_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Restrict_use.csv
Units_use.csv /content/drive/MyDrive/bcg/staging_files/ csv /content/drive/MyDrive/bcg/staging_files/Units_use.csv


In [ ]:
primary_person_use_df.show(10, False)

+--------+--------+--------+------------+------------------+-------------------------+--------+-----------------+------------+------------+-------------------+-----------------+--------------+-----------+---------------------+----------------+------------------+---------------------+----------------+-----------------+---------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+----------------------+-----------------+------------------+--------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|PRSN_TYPE_ID|PRSN_OCCPNT_POS_ID|PRSN_INJRY_SEV_ID        |PRSN_AGE|PRSN_ETHNICITY_ID|PRSN_GNDR_ID|PRSN_EJCT_ID|PRSN_REST_ID       |PRSN_AIRBAG_ID   |PRSN_HELMET_ID|PRSN_SOL_FL|PRSN_ALC_SPEC_TYPE_ID|PRSN_ALC_RSLT_ID|PRSN_BAC_TEST_RSLT|PRSN_DRG_SPEC_TYPE_ID|PRSN_DRG_RSLT_ID|DRVR_DRG_CAT_1_ID|PRSN_DEATH_TIME|INCAP_INJRY_CNT|NONINCAP_INJRY_CNT|POSS_INJRY_CNT|NON_INJRY_CNT|UNKN_INJRY_CNT|TOT_INJRY_CNT|DEATH_CNT|DRVR_LIC_TYPE_ID      |DRVR_LIC_STATE_ID|DRVR_LIC_CLS_

In [22]:
lcp = [colms.lower() for colms in primary_person_use_df.columns]
lcu = [colms.lower() for colms in units_use_df.columns]

In [ ]:
print(lcp)

['crash_id', 'unit_nbr', 'prsn_nbr', 'prsn_type_id', 'prsn_occpnt_pos_id', 'prsn_injry_sev_id', 'prsn_age', 'prsn_ethnicity_id', 'prsn_gndr_id', 'prsn_ejct_id', 'prsn_rest_id', 'prsn_airbag_id', 'prsn_helmet_id', 'prsn_sol_fl', 'prsn_alc_spec_type_id', 'prsn_alc_rslt_id', 'prsn_bac_test_rslt', 'prsn_drg_spec_type_id', 'prsn_drg_rslt_id', 'drvr_drg_cat_1_id', 'prsn_death_time', 'incap_injry_cnt', 'nonincap_injry_cnt', 'poss_injry_cnt', 'non_injry_cnt', 'unkn_injry_cnt', 'tot_injry_cnt', 'death_cnt', 'drvr_lic_type_id', 'drvr_lic_state_id', 'drvr_lic_cls_id', 'drvr_zip']


In [23]:
print(lcu)

['crash_id', 'unit_nbr', 'unit_desc_id', 'veh_parked_fl', 'veh_hnr_fl', 'veh_lic_state_id', 'vin', 'veh_mod_year', 'veh_color_id', 'veh_make_id', 'veh_mod_id', 'veh_body_styl_id', 'emer_respndr_fl', 'ownr_zip', 'fin_resp_proof_id', 'fin_resp_type_id', 'veh_dmag_area_1_id', 'veh_dmag_scl_1_id', 'force_dir_1_id', 'veh_dmag_area_2_id', 'veh_dmag_scl_2_id', 'force_dir_2_id', 'veh_inventoried_fl', 'veh_transp_name', 'veh_transp_dest', 'contrib_factr_1_id', 'contrib_factr_2_id', 'contrib_factr_p1_id', 'veh_trvl_dir_id', 'first_harm_evt_inv_id', 'incap_injry_cnt', 'nonincap_injry_cnt', 'poss_injry_cnt', 'non_injry_cnt', 'unkn_injry_cnt', 'tot_injry_cnt', 'death_cnt']


In [ ]:

primary_person_use_df.select(count('*'),count(col('crash_id')),
                             countDistinct(col('crash_id')),
                             sum(when(lower(col('prsn_injry_sev_id')) == 'killed',1).otherwise(0)),

                             sum(col('death_cnt'))
                             ).show(10, False)

+--------+---------------+------------------------+--------------------------------------------------------------------+--------------+
|count(1)|count(crash_id)|count(DISTINCT crash_id)|sum(CASE WHEN (lower(prsn_injry_sev_id) = killed) THEN 1 ELSE 0 END)|sum(death_cnt)|
+--------+---------------+------------------------+--------------------------------------------------------------------+--------------+
|156954  |156954         |83805                   |246                                                                 |246           |
+--------+---------------+------------------------+--------------------------------------------------------------------+--------------+



In [19]:
# Analysis 1
primary_person_use_df.where((lower(col('prsn_injry_sev_id')) == 'killed') & (lower(col('prsn_gndr_id')) == 'male'))\
.select(col('crash_id')).distinct().count()

180

In [100]:
#Analysis2
two_whlr_booked_df = units_use_df.where(upper(col('veh_body_styl_id')) == 'MOTORCYCLE').select(countDistinct('crash_id').alias('two_whlr_booked'))
two_whlr_booked_df.show(10, False)

+---------------+
|two_whlr_booked|
+---------------+
|754            |
+---------------+



In [21]:
# Analysis 3

fem_crash_state_df = primary_person_use_df.where((lower(col('prsn_gndr_id')) == 'female') 
                  & (~ lower(col('drvr_lic_state_id')).isin('na', 'unknown')))\
                    .groupBy(lower(col('drvr_lic_state_id')).alias('drvr_lic_state_id'))\
                      .agg(countDistinct(col('crash_id')).alias("num_crashes")).orderBy(col("num_crashes").desc())

window_crash_rank = Window.orderBy(col('num_crashes').desc())

max_female_crash_state_df =fem_crash_state_df.withColumn("rank", rank().over(window_crash_rank))\
.where("rank = 1").select(col('drvr_lic_state_id').alias("max_female_crash_state"))

max_female_crash_state_df.show(10, False)

+----------------------+
|max_female_crash_state|
+----------------------+
|texas                 |
+----------------------+



In [40]:
#Analysis4

win_spec_veh_inj_cnt = Window.orderBy(col("veh_inj_cnt").desc())
top_veh_make_id = units_use_df.where(lower(col('veh_make_id')) != 'na').groupBy("veh_make_id").agg(sum(col('tot_injry_cnt')).alias("veh_inj_cnt"))\
      .withColumn("rank", dense_rank().over(win_spec_veh_inj_cnt)).select(col('veh_make_id'), col('rank')).where("rank >=5 and rank <= 15")

top_veh_make_id.show(100, False)

+-----------+----+
|veh_make_id|rank|
+-----------+----+
|NISSAN     |5   |
|HONDA      |6   |
|GMC        |7   |
|HYUNDAI    |8   |
|KIA        |9   |
|JEEP       |10  |
|CHRYSLER   |11  |
|MAZDA      |12  |
|VOLKSWAGEN |13  |
|PONTIAC    |14  |
|LEXUS      |15  |
+-----------+----+



In [56]:
#Analysis5
#prsn_ethnicity_id shouldnot be na, unknown etc
primary_person_use_fil_df = primary_person_use_df.where(~upper(col('prsn_ethnicity_id')).isin('NA','UNKNOWN'))
#win_spec_veh_sty_eth = Window.partitionBy(col('veh_body_styl_id')).orderBy()
comp_df = units_use_df.where(~lower(col('veh_body_styl_id')).isin('na', 'not reported' ,'unknown'))\
      .join(primary_person_use_fil_df, "crash_id", "inner")\
      .select(col('crash_id'),
              upper(col('veh_body_styl_id')).alias('veh_body_styl_id'),
              upper(col('prsn_ethnicity_id')).alias('prsn_ethnicity_id'))
#comp_df.show(10, False)

agg_comp_df = comp_df.groupBy(col('veh_body_styl_id'), col('prsn_ethnicity_id')).agg(countDistinct(col('crash_id')).alias('cnt_dis_crash_id')).orderBy(col('veh_body_styl_id'))

#agg_comp_df.show(100, False)

win_spec_veh_sty_eth = Window.partitionBy(col('veh_body_styl_id')).orderBy(col('cnt_dis_crash_id').desc())

top_prsn_ethnicity_id_df = agg_comp_df.withColumn("rank", dense_rank().over(win_spec_veh_sty_eth))\
      .where(col('rank') == 1)\
      .select(col('veh_body_styl_id'), col('prsn_ethnicity_id').alias('top_prsn_ethnicity_id'))

top_prsn_ethnicity_id_df.show(100, False)

+---------------------------------+---------------------+
|veh_body_styl_id                 |top_prsn_ethnicity_id|
+---------------------------------+---------------------+
|AMBULANCE                        |WHITE                |
|BUS                              |HISPANIC             |
|FARM EQUIPMENT                   |WHITE                |
|FIRE TRUCK                       |WHITE                |
|MOTORCYCLE                       |WHITE                |
|NEV-NEIGHBORHOOD ELECTRIC VEHICLE|WHITE                |
|OTHER  (EXPLAIN IN NARRATIVE)    |WHITE                |
|PASSENGER CAR, 2-DOOR            |WHITE                |
|PASSENGER CAR, 4-DOOR            |WHITE                |
|PICKUP                           |WHITE                |
|POLICE CAR/TRUCK                 |WHITE                |
|POLICE MOTORCYCLE                |HISPANIC             |
|SPORT UTILITY VEHICLE            |WHITE                |
|TRUCK                            |WHITE                |
|TRUCK TRACTOR

In [110]:
#Analysis 6
cntns_car = instr(upper(col('veh_body_styl_id')) , 'CAR') >=1
units_cntns_car_df = units_use_df.withColumn('cntns_car_flag', cntns_car)\
                    .where(col('cntns_car_flag')).select(col('crash_id')).distinct()

win_spec_al_zip = Window.orderBy(col('num_tot_crashes').desc())
alc_prsn_df = primary_person_use_df.join(units_cntns_car_df, "crash_id", "inner").where((upper(col('prsn_alc_rslt_id')) == 'POSITIVE') & (col('drvr_zip').isNotNull()))\
              .groupBy(col('drvr_zip')).agg(countDistinct('crash_id').alias('num_tot_crashes'))\
              .withColumn("rank", rank().over(win_spec_al_zip))

top_zip_al_unit_df = alc_prsn_df.where(col('rank')<=5)\
                      .select(col('drvr_zip'), col('rank'))

top_zip_al_unit_df.show(10, False)


+--------+---------------+----+
|drvr_zip|num_tot_crashes|rank|
+--------+---------------+----+
|78521   |41             |1   |
|76010   |33             |2   |
|79936   |31             |3   |
|79938   |29             |4   |
|78550   |26             |5   |
|78240   |26             |5   |
|79912   |25             |7   |
|78223   |23             |8   |
|78741   |23             |8   |
|78201   |22             |10  |
|79928   |22             |10  |
|78233   |21             |12  |
|78230   |21             |12  |
|78753   |21             |12  |
|78228   |20             |15  |
|79924   |20             |15  |
|79907   |20             |15  |
|75052   |20             |15  |
|76017   |19             |19  |
|78660   |18             |20  |
|77449   |18             |20  |
|78207   |17             |22  |
|75067   |17             |22  |
|76014   |17             |22  |
|78237   |17             |22  |
|78210   |17             |22  |
|76018   |17             |22  |
|78251   |16             |28  |
|75050  

In [80]:
#Analysis 7
cntns_ins = instr(col('fin_resp_type_id') , 'INSURANCE') >=1

units_use_ins = units_use_df\
        .withColumn('veh_dmg_lvl_1',regexp_extract(col('veh_dmag_scl_1_id'), r'(\d+)', 1).cast('bigint'))\
        .withColumn('veh_dmg_lvl_2',regexp_extract(col('veh_dmag_scl_2_id'), r'(\d+)', 1).cast('bigint'))\
        .withColumn('type_id_ins_flg', cntns_ins)\
        .select(col('crash_id'),col('veh_dmg_lvl_1'), col('veh_dmg_lvl_2'), col('type_id_ins_flg'))\
        .where((~col('fin_resp_proof_id').isin('NA','NR')) & ((col('veh_dmg_lvl_1') >4) | (col('veh_dmg_lvl_2') >4) ) & (col('type_id_ins_flg')))\
        
no_dmg_df = units_use_ins.join(damages_use_df, "crash_id", "leftanti")

cnt_no_dmg = no_dmg_df.select(countDistinct(col('crash_id')).alias('cnt_no_dmg'))

cnt_no_dmg.show(10, False)

+----------+
|cnt_no_dmg|
+----------+
|8849      |
+----------+



In [91]:
#Analysis 8

top_vhl_clr = units_use_df.where(upper(col('veh_color_id')) != 'NA').groupBy(col('veh_color_id')).agg(countDistinct(col('crash_id')).alias('count_clr')).orderBy(col("count_clr").desc()).limit(10)

+------------+---------+
|veh_color_id|count_clr|
+------------+---------+
|WHI         |31095    |
|BLK         |24597    |
|SIL         |18934    |
|GRY         |16614    |
|BLU         |14071    |
|RED         |12840    |
|GRN         |6474     |
|MAR         |5745     |
|TAN         |4672     |
|GLD         |3932     |
+------------+---------+



In [94]:
top_state_ofnces = primary_person_use_df.where(~upper(col('drvr_lic_state_id')).isin('NA', 'UNKNOWN','OTHER'))\
          .groupBy(col('drvr_lic_state_id')).agg(countDistinct('crash_id').alias('count_crsh')).orderBy(col("count_crsh").desc())\
          .select(col('drvr_lic_state_id')).limit(25)
top_state_ofnces.show(100, False)

+-----------------+
|drvr_lic_state_id|
+-----------------+
|Texas            |
|Mexico           |
|Louisiana        |
|New Mexico       |
|California       |
|Florida          |
|Oklahoma         |
|Arkansas         |
|Arizona          |
|Georgia          |
|Colorado         |
|Illinois         |
|Missouri         |
|Tennessee        |
|Mississippi      |
|North Carolina   |
|Kansas           |
|Alabama          |
|Michigan         |
|Washington       |
|Ohio             |
|New York         |
|Virginia         |
|South Carolina   |
|Pennsylvania     |
+-----------------+



In [97]:
df_oz = units_use_df.select(col('ownr_zip'), col('veh_lic_state_id')).distinct().select(count('*'), countDistinct('ownr_zip'), countDistinct('veh_lic_state_id')).show(100, False)

+--------+------------------------+--------------------------------+
|count(1)|count(DISTINCT ownr_zip)|count(DISTINCT veh_lic_state_id)|
+--------+------------------------+--------------------------------+
|10792   |6367                    |59                              |
+--------+------------------------+--------------------------------+



In [98]:
units_use_df.select(col('ownr_zip'), col('veh_lic_state_id')).distinct().groupBy('ownr_zip').agg(countDistinct('veh_lic_state_id').alias('c')).where("c>1").show(100, False)

+----------+---+
|ownr_zip  |c  |
+----------+---+
|77303     |3  |
|89106     |2  |
|77371     |2  |
|77339     |6  |
|91910     |2  |
|73463     |2  |
|78382     |3  |
|75602     |3  |
|79849     |5  |
|75007     |3  |
|78073     |2  |
|32812     |2  |
|79360     |4  |
|79013     |2  |
|91767     |2  |
|79345     |2  |
|76673     |3  |
|78211     |2  |
|95212     |2  |
|78022     |2  |
|37160     |3  |
|33615     |2  |
|79504     |2  |
|48088     |2  |
|77872     |2  |
|23462     |6  |
|73036     |2  |
|29401     |2  |
|75766     |2  |
|95834     |2  |
|52406     |2  |
|88295     |2  |
|62353     |2  |
|77375     |3  |
|66062     |3  |
|55432     |2  |
|79313     |2  |
|72110     |2  |
|75763     |2  |
|39218     |3  |
|7054      |4  |
|77302     |4  |
|77871     |2  |
|39301     |2  |
|77062     |4  |
|84332     |2  |
|67202     |2  |
|77471     |4  |
|79705     |9  |
|22701-7989|2  |
|63021     |2  |
|77364     |4  |
|55014     |2  |
|75219     |4  |
|71202     |2  |
|75169     |2 

In [92]:
primary_person_use_df.select(col('drvr_lic_state_id')).distinct().show(100, False)

+------------------------+
|drvr_lic_state_id       |
+------------------------+
|Utah                    |
|Hawaii                  |
|U.S. Virgin Islands     |
|Minnesota               |
|Ohio                    |
|Northern Mariana Islands|
|Arkansas                |
|Oregon                  |
|NA                      |
|Texas                   |
|North Dakota            |
|Pennsylvania            |
|Connecticut             |
|Nebraska                |
|Vermont                 |
|Nevada                  |
|Puerto Rico             |
|Washington              |
|Illinois                |
|Oklahoma                |
|Unknown                 |
|District of Columbia    |
|Other                   |
|Delaware                |
|American Samoa (Islands)|
|Alaska                  |
|New Mexico              |
|West Virginia           |
|Missouri                |
|Rhode Island            |
|Mexico                  |
|Georgia                 |
|Montana                 |
|Michigan                |
|

In [63]:
units_use_df.select(col('fin_resp_proof_id'), col('fin_resp_type_id')).distinct().show(100, False)

+-----------------+----------------------------------------+
|fin_resp_proof_id|fin_resp_type_id                        |
+-----------------+----------------------------------------+
|2                |SURETY BOND                             |
|2                |PROOF OF LIABILITY INSURANCE            |
|1                |PROOF OF LIABILITY INSURANCE            |
|NR               |NA                                      |
|3                |CERTIFICATE OF SELF-INSURANCE           |
|1                |LIABILITY INSURANCE POLICY              |
|2                |NA                                      |
|3                |NA                                      |
|2                |CERTIFICATE OF SELF-INSURANCE           |
|4                |INSURANCE BINDER                        |
|3                |PROOF OF LIABILITY INSURANCE            |
|3                |INSURANCE BINDER                        |
|4                |CERTIFICATE OF DEPOSIT WITH COUNTY JUDGE|
|2                |CERTI

In [53]:
primary_person_use_df.select(col('prsn_ethnicity_id')).distinct().show(100, False)

+---------------------------+
|prsn_ethnicity_id          |
+---------------------------+
|WHITE                      |
|BLACK                      |
|HISPANIC                   |
|NA                         |
|AMER. INDIAN/ALASKAN NATIVE|
|UNKNOWN                    |
|OTHER                      |
|ASIAN                      |
+---------------------------+



In [41]:


units_use_df.where(~lower(col('veh_body_styl_id')).isin('na', 'not reported' ,'unknown'))\
      .select(col('veh_body_styl_id')).distinct().show(100, False)

+---------------------------------+
|veh_body_styl_id                 |
+---------------------------------+
|BUS                              |
|NA                               |
|VAN                              |
|PICKUP                           |
|SPORT UTILITY VEHICLE            |
|PASSENGER CAR, 4-DOOR            |
|FIRE TRUCK                       |
|TRUCK                            |
|UNKNOWN                          |
|AMBULANCE                        |
|POLICE CAR/TRUCK                 |
|MOTORCYCLE                       |
|YELLOW SCHOOL BUS                |
|POLICE MOTORCYCLE                |
|PASSENGER CAR, 2-DOOR            |
|TRUCK TRACTOR                    |
|FARM EQUIPMENT                   |
|NEV-NEIGHBORHOOD ELECTRIC VEHICLE|
|OTHER  (EXPLAIN IN NARRATIVE)    |
|NOT REPORTED                     |
+---------------------------------+



In [24]:
primary_person_use_df.select(count("*"), countDistinct(col('crash_id')), sum(col('tot_injry_cnt'))).show(10, False)

+--------+------------------------+------------------+
|count(1)|count(DISTINCT crash_id)|sum(tot_injry_cnt)|
+--------+------------------------+------------------+
|156954  |83805                   |33276             |
+--------+------------------------+------------------+



In [25]:
primary_person_use_df.dropDuplicates().select(count("*"), countDistinct(col('crash_id')), sum(col('tot_injry_cnt'))).show(10, False)

+--------+------------------------+------------------+
|count(1)|count(DISTINCT crash_id)|sum(tot_injry_cnt)|
+--------+------------------------+------------------+
|156954  |83805                   |33276             |
+--------+------------------------+------------------+



In [26]:
primary_person_use_df.orderBy("crash_id").show(100, False)

+--------+--------+--------+---------------------------------+------------------+-------------------------+--------+-----------------+------------+--------------+-------------------+-----------------+-----------------+-----------+---------------------+----------------+------------------+---------------------+----------------+-----------------+---------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+----------------------+-----------------+------------------+--------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|PRSN_TYPE_ID                     |PRSN_OCCPNT_POS_ID|PRSN_INJRY_SEV_ID        |PRSN_AGE|PRSN_ETHNICITY_ID|PRSN_GNDR_ID|PRSN_EJCT_ID  |PRSN_REST_ID       |PRSN_AIRBAG_ID   |PRSN_HELMET_ID   |PRSN_SOL_FL|PRSN_ALC_SPEC_TYPE_ID|PRSN_ALC_RSLT_ID|PRSN_BAC_TEST_RSLT|PRSN_DRG_SPEC_TYPE_ID|PRSN_DRG_RSLT_ID|DRVR_DRG_CAT_1_ID|PRSN_DEATH_TIME|INCAP_INJRY_CNT|NONINCAP_INJRY_CNT|POSS_INJRY_CNT|NON_INJRY_CNT|UNKN_INJRY_CNT|TOT_INJRY_CNT|DEATH_CNT|DR